In [ ]:
f = 5
f 

In [2]:
from PIL import Image
from tor

In [ ]:
chvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch
from skimage import io
import numpy as np

In [3]:
im = io.imread('/content/drive/MyDrive/1.tif')

im = torch.from_numpy(im.astype(np.float32))
im.shape #frames,height,width

torch.Size([512, 512, 1024])

In [4]:
im.unsqueeze_(0).shape #null(C),frames,height,width

torch.Size([1, 512, 512, 1024])

In [5]:
data = [im for i in range(4)]
data = torch.stack(data)
data.shape #batch_size,null(C),frames,height,width

torch.Size([4, 1, 512, 512, 1024])

In [7]:
ICP = [30,40,50,60]

In [8]:
batch = [data]+[ICP]
len(batch)

2

In [43]:
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

class CNN(torch.nn.Module):

  def __init__(self):
    
    super(CNN, self).__init__()
    
    ######################################################
    #inputs###############################################
    self.c1_out = 2                                     ##
                                                        ##
    self.c2_out = 15                                    ##
                                                        ##
    self.c3_out = 12                                    ##
                                                        ##
    self.c4_out = 5                                     ##
    ######################################################


    #1st layer ###########################################
    self.conv1 = nn.Conv3d(in_channels = 1, out_channels = self.c1_out,\
                           kernel_size = (5,5,5), stride=(30,30,30), padding=0) ## frames 
    #relu

    
    ######################################################


    #Linear layer ########################################
    self.fc1 = nn.Linear(2* 17* 17* 34, 1)
    #relu
    ######################################################

  def forward(self, x):
    out = F.relu(self.conv1(x))
    print('-----')
    print(out.shape)
    out = out.view(-1, out.size()[1] * out.size()[2] * out.size()[3] * out.size()[4])
    out = F.relu(self.fc1(out))
    print('-----')
    print(out.shape)
    return out

In [49]:
from tqdm.notebook import tqdm

def get_loss_and_correct(model, batch, criterion, device):
  # Implement forward pass and loss calculation for one batch.
  # Remember to move the batch to device.
  # 
  # Return a tuple:
  # - loss for the batch (Tensor)
  # - number of correctly classified examples in the batch (Tensor)

  imgs = batch[0]

  labels = batch[1]

  outputs = model(imgs)
  print(outputs)
  print(labels)
  loss = criterion(outputs,torch.tensor(labels))
  print(loss)
  preds = outputs.data.max(1, keepdim=True)[1]

  return (loss,torch.eq(preds.flatten(),labels).sum())


def step(loss, optimizer):
  # Implement backward pass and update.
  # TODO
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  
model = CNN()

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()

total_train_loss = 0.0
total_train_correct = 0.0
  
loss, correct = get_loss_and_correct(model, batch, criterion, device = None)
step(loss, optimizer)
total_train_loss += loss.item()
total_train_correct += correct.item()

-----
torch.Size([4, 2, 17, 17, 34])
-----
torch.Size([4, 1])
tensor([[13.0931],
        [13.0931],
        [13.0931],
        [13.0931]], grad_fn=<ReluBackward0>)
[30, 40, 50, 60]
tensor(1143.0482, grad_fn=<MseLossBackward>)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: ignored